In [1]:
import math
import numpy as np
import pandas as pd

In [2]:
fdata = pd.read_csv('data_histories.csv', index_col="dp_folio")
fdata.head()

,id_sexo,Aedad,AAedad,Apuesto,id_gestud,AIMC,fuma,fuma_act,ejer_act,ejer1,...,locout5,locout10,locout20,locout30,rest_act,rest1,rest5,rest10,rest20,rest30
dp_folio,,,,,,,,,,,,,,,,,,,,,
1,F,51,6,Admin,CarTec,4,1,3,0,2,...,1,1,1,1,0,0,0,0,0,0
2,F,38,4,Sec,Bach,3,2,-1,0,0,...,0,0,0,0,1,3,0,0,0,0
3,F,34,3,Int,Sec,5,1,1,0,0,...,1,-1,-1,-1,0,0,0,-1,-1,-1
4,M,63,8,Jef,CarTec,4,2,-1,2,2,...,0,0,0,-1,0,0,0,0,0,0
5,M,42,4,EM,Sec,3,1,2,2,2,...,0,0,0,0,0,0,0,0,0,0


### Functions for clusterization

These functions will put data into "buckets".

In [4]:
#Function to clusterize categories of a certain feature, and add the new clusterized feature as a new column
# Clusters should be an input of the form {cluster_A: {categories}, cluster_B: [categories]}
# Ex. obesity = {0:[1,2,3], 1:[4,5,6]}
def clusterizeDiscrete(feature, clusters, new_name, data):
    new_data = data.copy()
    original_list = data.loc[1:1080, feature]
    new_list = []
    for index in original_list.index:
        cat = False
        if original_list[index] == -1 or original_list[index] == "-1":
            new_list.append("N")
            continue
        for cluster in clusters:
            if original_list[index] in clusters[cluster]:
                new_list.append(cluster)
                cat = True
        if cat == False:
            new_list.append("N")
    new_data[new_name] = new_list
    return new_data

# Function to clusterize categories of a certain continous feature, and add the new clusterized feature as a
# new column
# Clusters shoud be an input of the form {cluster_A: {lambdaFunction1}, cluster_B: lambdaFunction2}
def clusterizeContinuous(feature, clusters, new_name, data):
    new_data = data.copy()
    original_list = data.loc[1:1080, feature]
    new_list = []
    for index in original_list.index:
        if original_list[index] == -1 or original_list[index] == "-1":
            new_list.append("N")
            continue
        for cluster in clusters:
            if eval(clusters[cluster])(original_list[index]):
                new_list.append(cluster)
                break
    new_data[new_name] = new_list
    return new_data
        

# Set of auxiliary high-order functions that will evaluate the conditions to binarize a history
def lessThan(num):
    return lambda n: n < num

def lessQThan(num):
    return lambda n: n <= num

def greaterThan(num):
    return lambda n: n > num

def greaterQThan(num):
    return lambda n: n >= num

def between(num1, num2):
    return lambda n: n <= num2 and n >= num1

### Clusters definition and creation

In [5]:
# Clusterize between obese and non obese
obesity = {0:[1,2,3], 1:[4,5,6]}
ndata = clusterizeDiscrete("AIMC", obesity, "obesity", fdata)

# Clusterize between degrees of study (higher degree vs. non higher degree)
degree = {0:["Prim", "Sec", "Bach", "CarTec", "Otro"], 1:["Lic", "Mast", "Doc", "PDoc"]}
ndata = clusterizeDiscrete("id_gestud", degree, "hdegree", ndata)

# Clusterize excercise features
ejer = {"A":"greaterQThan(2.5)", "B":"lessThan(2.5)"}
ndata = clusterizeContinuous("ejer_act", ejer, "ejer0B", ndata)
ndata = clusterizeContinuous("ejer1", ejer, "ejer1B", ndata)
ndata = clusterizeContinuous("ejer5", ejer, "ejer5B", ndata)
ndata = clusterizeContinuous("ejer10", ejer, "ejer10B", ndata)
ndata = clusterizeContinuous("ejer20", ejer, "ejer20B", ndata)
ndata = clusterizeContinuous("ejer30", ejer, "ejer30B", ndata)

# Clusterize stress features
estres = {"A":[4,5], "B":[1,2,3]}
ndata = clusterizeDiscrete("estres_act", estres, "estres0B", ndata)
ndata = clusterizeDiscrete("estres1", estres, "estres1B", ndata)
ndata = clusterizeDiscrete("estres5", estres, "estres5B", ndata)
ndata = clusterizeDiscrete("estres10", estres, "estres10B", ndata)
ndata = clusterizeDiscrete("estres20", estres, "estres20B", ndata)
ndata = clusterizeDiscrete("estres30", estres, "estres30B", ndata)

# Clusterize weight features
peso = {"A":[4,5], "B":[1,2,3]}
ndata = clusterizeDiscrete("peso_act", peso, "peso0B", ndata)
ndata = clusterizeDiscrete("peso1", peso, "peso1B", ndata)
ndata = clusterizeDiscrete("peso5", peso, "peso5B", ndata)
ndata = clusterizeDiscrete("peso10", peso, "peso10B", ndata)
ndata = clusterizeDiscrete("peso20", peso, "peso20B", ndata)
ndata = clusterizeDiscrete("peso30", peso, "peso30B", ndata)

# Clusterize weight features
condi = {"A":[4,5], "B":[1,2,3]}
ndata = clusterizeDiscrete("condi_act", condi, "condi0B", ndata)
ndata = clusterizeDiscrete("condi1", condi, "condi1B", ndata)
ndata = clusterizeDiscrete("condi5", condi, "condi5B", ndata)
ndata = clusterizeDiscrete("condi10", condi, "condi10B", ndata)
ndata = clusterizeDiscrete("condi20", condi, "condi20B", ndata)
ndata = clusterizeDiscrete("condi30", condi, "condi30B", ndata)

# Clusterize health features
health = {"A":[4,5], "B":[1,2,3]}
ndata = clusterizeDiscrete("salud_act", health, "salud0B", ndata)
ndata = clusterizeDiscrete("salud1", health, "salud1B", ndata)
ndata = clusterizeDiscrete("salud5", health, "salud5B", ndata)
ndata = clusterizeDiscrete("salud10", health, "salud10B", ndata)
ndata = clusterizeDiscrete("salud20", health, "salud20B", ndata)
ndata = clusterizeDiscrete("salud30", health, "salud30B", ndata)

# Clusterize job features
academic = {0:["Admin", "Asi", "Coo", "E", "ED", "EM", "Int", "Jef", "Lab", "Sec", "Tec", "Vig"], 1:["Acade", "Inv", "InvE"]}
ndata = clusterizeDiscrete("Apuesto", academic, "academic", ndata)

# Clusterize walking features
walking = {"A":"greaterQThan(1800.0)", "B":"lessThan(1800.0)"}
ndata = clusterizeContinuous("dis_dia", walking, "dis_dia0B", ndata)
ndata = clusterizeContinuous("dis1_dia", walking, "dis_dia1B", ndata)
ndata = clusterizeContinuous("dis5_dia", walking, "dis_dia5B", ndata)
ndata = clusterizeContinuous("dis10_dia", walking, "dis_dia10B", ndata)
ndata = clusterizeContinuous("dis20_dia", walking, "dis_dia20B", ndata)
ndata = clusterizeContinuous("dis30_dia", walking, "dis_dia30B", ndata)

In [6]:
ndata.head()

,id_sexo,Aedad,AAedad,Apuesto,id_gestud,AIMC,fuma,fuma_act,ejer_act,ejer1,...,salud10B,salud20B,salud30B,academic,dis_dia0B,dis_dia1B,dis_dia5B,dis_dia10B,dis_dia20B,dis_dia30B
dp_folio,,,,,,,,,,,,,,,,,,,,,
1,F,51,6,Admin,CarTec,4,1,3,0,2,...,A,A,A,0,B,B,A,N,N,N
2,F,38,4,Sec,Bach,3,2,-1,0,0,...,A,A,A,0,A,B,B,B,A,B
3,F,34,3,Int,Sec,5,1,1,0,0,...,A,A,N,0,B,B,B,B,B,B
4,M,63,8,Jef,CarTec,4,2,-1,2,2,...,A,A,A,0,A,A,A,A,A,A
5,M,42,4,EM,Sec,3,1,2,2,2,...,B,N,N,0,B,B,B,N,N,N


### Functions for grouping variables

These functions will aid in the construction of histories and other composite random variables.

In [7]:
# Function to build a composite random variable from two variables
def groupVariables(feature_1, feature_2, new_name, data):
    new_data = data.copy()
    list_feat_1 = data.loc[1:1080, feature_1]
    list_feat_2 = data.loc[1:1080, feature_2]
    new_list = []
    for index in list_feat_1.index:
        new_list.append(str(list_feat_1[index]) + str(list_feat_2[index]))
    new_data[new_name] = new_list
    return new_data

# Function to build a composite random variable from several variables (to create a history, for example)
def groupListOfVariables(list_of_features, new_name, data):
    new_data = data.copy()
    new_list = []
    i = 0
    for feature in list_of_features:
        buffer_list = data.loc[1:1080, feature]
        if i == 0:
            for index in buffer_list.index:
                new_list.append(str(buffer_list[index]))
                i = i + 1
        else:
            k = 0
            for index in buffer_list.index:
                new_list[k] = new_list[k] + str(buffer_list[index])
                k = k + 1
    new_data[new_name] = new_list
    return new_data

In [63]:
#test = groupListOfVariables(["ejer0B", "ejer1B", "ejer5B", "ejer10B", "ejer20B", "ejer30B"], "ExerHistory", ndata)
#test.head()

### Histories creation

In [8]:
ndata = groupListOfVariables(["ejer0B", "ejer1B", "ejer5B", "ejer10B", "ejer20B", "ejer30B"], "ejerHistory", ndata)
ndata = groupListOfVariables(["condi0B", "condi1B", "condi5B", "condi10B", "condi20B", "condi30B"], "condiHistory", ndata)
ndata = groupListOfVariables(["estres0B", "estres1B", "estres5B", "estres10B", "estres20B", "estres30B"], "estresHistory", ndata)
ndata = groupListOfVariables(["salud0B", "salud1B", "salud5B", "salud10B", "salud20B", "salud30B"], "saludHistory", ndata)
ndata = groupListOfVariables(["dis_dia0B", "dis_dia1B", "dis_dia5B", "dis_dia10B", "dis_dia20B", "dis_dia30B"], "dis_diaHistory", ndata)
#ndata = groupListOfVariables(["peso0B", "peso1B", "peso5B", "peso10B", "peso20B", "peso30B"], "pesoHistory", ndata)
ndata.head()

,id_sexo,Aedad,AAedad,Apuesto,id_gestud,AIMC,fuma,fuma_act,ejer_act,ejer1,...,dis_dia1B,dis_dia5B,dis_dia10B,dis_dia20B,dis_dia30B,ejerHistory,condiHistory,estresHistory,saludHistory,dis_diaHistory
dp_folio,,,,,,,,,,,,,,,,,,,,,
1,F,51,6,Admin,CarTec,4,1,3,0,2,...,B,A,N,N,N,BBBBBA,BAAAAA,ABANNB,AAAAAA,BBANNN
2,F,38,4,Sec,Bach,3,2,-1,0,0,...,B,B,B,A,B,BBAAAB,AAAAAA,BBABBB,AABAAA,ABBBAB
3,F,34,3,Int,Sec,5,1,1,0,0,...,B,B,B,B,B,BBBANN,BBBBAN,AAAABN,BBAAAN,BBBBBB
4,M,63,8,Jef,CarTec,4,2,-1,2,2,...,A,A,A,A,A,BBAAAA,BBAAAA,BABBBN,AAAAAA,AAAAAA
5,M,42,4,EM,Sec,3,1,2,2,2,...,B,B,N,N,N,BBBBBB,BBAANN,BBBANN,AABBNN,BBBNNN


### Functions for manipulating histories

In [9]:
# Function to determine if a certain history matches a general pattern
def matchStructure(history, structure):
    if type(history) is str: 
        if len(history) != len(structure):
            return False
        for i in range(len(structure)):
            if history[i] != structure[i] and structure[i] != '*':
                return False
        return True
    else:
        return False

In [10]:
matchStructure(1, "****")

False

### Functions for counting/calculating probabilities

In [11]:
# Functions to count ocurrences for a category of a feature (NX)

# Count the number of instances inside the database whose feature X = category
# Ex: getNX('AIMC', 3, fdata)
def getNX(feature, category, data):
    count = 0
    for index in data.index:
        if data.loc[index][feature] == category:
            count = count + 1
        elif matchStructure(data.loc[index][feature], category):
            count = count + 1
    return count

# Count the number of instances inside the database whose feature X_1 = category_1 and feature X_2 = category_2
# Ex: getNCX('AIMC', 3, 'Obesidad', 1, fdata)
def getNCX(feature_1, category_1, feature_2, category_2, data):
    count = 0
    for index in data.index:
        if data.loc[index][feature_1] == category_1 and data.loc[index][feature_2] == category_2:
            count = count + 1
        elif matchStructure(data.loc[index][feature_1], category_1) and data.loc[index][feature_2] == category_2:
            count = count + 1
        elif matchStructure(data.loc[index][feature_1], category_1) and matchStructure(data.loc[index][feature_2], category_2):
            count = count + 1
        elif data.loc[index][feature_1] == category_1 and matchStructure(data.loc[index][feature_2], category_2):
            count = count + 1
    return count

# Get a conditional probability P(F_1 = C_1 | F_2 = C_2)
def getCondProb(feature_1, category_1, feature_2, category_2, data):
    ncx = getNCX(feature_1, category_1, feature_2, category_2, data)
    nx = getNX(feature_2, category_2, data)
    if nx != 0:
        p = ncx / float(nx)
    else:
        p = 0
    #return {'P': p, 'nx': nx, 'ncx': ncx}
    return p

def getEpsilon(feature, category, classFeature, classCategory, data):
    n = len(data)
    nx = getNX(feature, category, data)
    nc = getNX(classFeature, classCategory, data)
    ncx = getNCX(feature, category, classFeature, classCategory, data)
    if n != 0 and nx != 0:
        pc = nc / float(n)
        pcx = ncx / float(nx)
        epsilon = nx * (pcx - pc) / math.sqrt(nx * pc * (1 - pc))
    else:
        epsilon = 0
    #print 'Epsilon :' + str(epsilon)
    return {'feat': feature, 'cat': category, 'class': classFeature, 'classcat': classCategory, 'epsilon': epsilon, 'nx': nx, 'ncx': ncx, 'nc': nc}

def getHistoriesEpsilons(feature, categories, classFeature, classCategory, data):
    epsilons = []
    for category in categories:
        epsilons.append(getEpsilon(feature, category, classFeature, classCategory, data))
    return epsilons

In [12]:
def stringGenerator(string, num):
    seeds = ["A", "B", "*"]
    strings = []
    if num == 0:
        return string
    else:
        for seed in seeds:
            strings.append(stringGenerator(string + seed, num - 1))
    return strings

def stringPadding(pre_padding, strings, post_padding):
    new_strings = []
    for string in strings:
        new_strings.append(pre_padding + string + post_padding)
    return new_strings

### Queries (Histories)

In [16]:
buf = np.array(stringGenerator("", 4)).flatten()
posibilities = stringPadding("", buf, "**")

In [360]:
his_epsilons = getHistoriesEpsilons("estresHistory", posibilities, "Apuesto", "Int", ndata)

In [369]:
df = pd.DataFrame(his_epsilons).set_index("cat")
df.sort_values('epsilon', ascending=False)

,class,classcat,epsilon,feat,nc,ncx,nx
cat,,,,,,,
*BBB**,Apuesto,Int,3.922740,estresHistory,110,64,395
*BB***,Apuesto,Int,3.407387,estresHistory,110,66,435
*B*B**,Apuesto,Int,3.228545,estresHistory,110,66,444
**BB**,Apuesto,Int,2.774743,estresHistory,110,88,651
*B****,Apuesto,Int,2.565140,estresHistory,110,71,521
**B***,Apuesto,Int,2.405282,estresHistory,110,92,710
***B**,Apuesto,Int,1.171356,estresHistory,110,98,857
*ABA**,Apuesto,Int,0.043634,estresHistory,110,2,19
******,Apuesto,Int,0.000000,estresHistory,110,110,1076


In [308]:
df.to_csv(path_or_buf="epsilons_saludHistory_obesity.csv")

In [13]:
def getHistoriesEpsilonsLOC(feature, categories, classFeature, data):
    catlist = getCategories(classFeature, data)
    epsilons = []
    for category in catlist:
        epsilons = epsilons + getHistoriesEpsilons(feature, categories, classFeature, category, data)
    return pd.DataFrame(epsilons)

In [17]:
job_st = getHistoriesEpsilonsLOC("estresHistory", posibilities, "Apuesto", ndata)

In [19]:
job_st.to_csv(path_or_buf="epsilons_estresHistory_jobs.csv")

### Queries (Other)

In [339]:
getEpsilon("Apuesto", "EM", "estresHistory", "BBB***", ndata)

{'cat': 'EM',
 'class': 'estresHistory',
 'classcat': 'BBB***',
 'epsilon': -2.4236798422802135,
 'feat': 'Apuesto',
 'nc': 358,
 'ncx': 14,
 'nx': 71}

In [14]:
def getAllEpsilons(feature, classFeature, classCategory, data):
    catlist = getCategories(feature, data)
    epsilons = []
    for category in catlist:
        epsilons.append(getEpsilon(feature, category, classFeature, classCategory, data))
    return pd.DataFrame(epsilons)

def getCategories(feature, data):
    index_list = data.loc[1:1080, feature]
    categories = {}
    for index in index_list.index:
        categories[index_list[index]] = "1"
    return categories.keys()
    

In [343]:
eps = getAllEpsilons("Apuesto", "estresHistory", "BBB***", ndata)

In [346]:
eps.sort_values("epsilon", ascending=False)

,cat,class,classcat,epsilon,feat,nc,ncx,nx
8,Int,estresHistory,BBB***,4.330680,Apuesto,358,58,110
7,Lab,estresHistory,BBB***,1.847083,Apuesto,358,22,48
2,Vig,estresHistory,BBB***,1.706208,Apuesto,358,16,34
6,Tec,estresHistory,BBB***,1.696567,Apuesto,358,25,57
13,InvE,estresHistory,BBB***,1.227593,Apuesto,358,2,3
3,Admin,estresHistory,BBB***,1.080403,Apuesto,358,29,74
10,Acade,estresHistory,BBB***,0.436334,Apuesto,358,81,234
11,Jef,estresHistory,BBB***,-0.420331,Apuesto,358,30,96
5,Inv,estresHistory,BBB***,-0.525002,Apuesto,358,26,85
9,Sec,estresHistory,BBB***,-0.594226,Apuesto,358,20,67


In [348]:
q2 = getAllEpsilons("Apuesto", "obesity", 1, ndata)

In [350]:
q2.sort_values("epsilon", ascending=False)

,cat,class,classcat,epsilon,feat,nc,ncx,nx
9,Sec,obesity,1,3.528588,Apuesto,228,26,67
2,Vig,obesity,1,2.851881,Apuesto,228,14,34
11,Jef,obesity,1,2.412116,Apuesto,228,30,96
7,Lab,obesity,1,2.058828,Apuesto,228,16,48
8,Int,obesity,1,2.027883,Apuesto,228,32,110
3,Admin,obesity,1,1.228810,Apuesto,228,20,74
13,InvE,obesity,1,0.514707,Apuesto,228,1,3
6,Tec,obesity,1,0.298819,Apuesto,228,13,57
12,Asi,obesity,1,0.185690,Apuesto,228,12,54
5,Inv,obesity,1,-0.533805,Apuesto,228,16,85
